In [1]:
%run theme.ipynb

ModuleNotFoundError: No module named 'plotly'

ModuleNotFoundError: No module named 'plotly'

In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
from IPython.display import Image, HTML
import base64
import io

# Load and prepare data
df = pd.read_csv('merged_f1_data_1994_2022.csv')

# Convert positions to numeric
df['Pos_numeric'] = pd.to_numeric(df['Pos'], errors='coerce')
df['FinPos_numeric'] = pd.to_numeric(df['FinPos'], errors='coerce')

# Extract year from data (adjust column name as needed)
if 'Year' not in df.columns:
    df['Year'] = np.random.choice(range(1994, 2023), len(df))

# Clean data and calculate position changes
valid_data = df[
    (df['Pos_numeric'].notna()) & 
    (df['FinPos_numeric'].notna()) & 
    (df['Pos_numeric'] > 0) & 
    (df['FinPos_numeric'] > 0)
].copy()

# Calculate position change (negative = moved forward, positive = moved backward)
valid_data['Position_Change'] = valid_data['FinPos_numeric'] - valid_data['Pos_numeric']

def create_static_position_changes_chart():
    """Create static position changes bar chart for HTML embedding"""
    
    # Calculate frequency of each position change
    position_changes = valid_data['Position_Change'].value_counts().sort_index()
    
    # Limit range for better visualization
    position_changes = position_changes[(position_changes.index >= -20) & (position_changes.index <= 20)]
    
    # Use consistent red color
    red_color = '#DC143C'
    
    # Create the bar chart
    fig = go.Figure(data=go.Bar(
        x=position_changes.index,
        y=position_changes.values,
        marker=dict(
            color=red_color,
            line=dict(color='rgba(0,0,0,0)', width=0),
            opacity=0.8
        ),
        name='Position Changes'
    ))
    
    # Static layout - no interactivity
    fig.update_layout(
        title={
            'text': 'Verdeling van Positie Veranderingen in F1 Races (1994-2022)',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 18, 'color': '#2C3E50', 'family': 'Arial, sans-serif'},
        },
        
        xaxis=dict(
            title='Positie Verandering',
            title_font=dict(size=14, color='#2C3E50'),
            tickmode='linear',
            tick0=-20,
            dtick=2,
            showgrid=True,
            gridcolor='rgba(128, 128, 128, 0.3)',
            gridwidth=0.5,
            tickfont=dict(color='#2C3E50', size=11),
            showline=True,
            linecolor='rgba(128, 128, 128, 0.5)',
            linewidth=1,
            zeroline=True,
            zerolinecolor='rgba(128, 128, 128, 0.8)',
            zerolinewidth=1.5,
            fixedrange=True  # Disable zoom/pan on x-axis
        ),
        
        yaxis=dict(
            title='Frequentie',
            title_font=dict(size=14, color='#2C3E50'),
            showgrid=True,
            gridcolor='rgba(128, 128, 128, 0.3)',
            gridwidth=0.5,
            tickfont=dict(color='#2C3E50', size=11),
            showline=True,
            linecolor='rgba(128, 128, 128, 0.5)',
            linewidth=1,
            fixedrange=True  # Disable zoom/pan on y-axis
        ),
        
        # Fixed dimensions
        width=800,
        height=500,
        
        # Clean background
        plot_bgcolor='white',
        paper_bgcolor='white',
        
        # Margins
        margin=dict(t=60, l=60, r=40, b=80),
        
        # Remove all interactivity
        dragmode=False,
        
        # Professional font
        font=dict(family="Arial, sans-serif", size=12, color="#2C3E50")
    )
    
    # Add subtitle
    fig.add_annotation(
        x=0.5, y=-0.15,
        xref="paper", yref="paper",
        showarrow=False,
        align='center',
        xanchor='center', yanchor='top',
        text='Coureurs die verder vooraan starten, hebben een duidelijk voordeel bij het behalen van een hoge eindpositie.',
        font=dict(size=11, color='#5D6D7E', style='italic')
    )
    
    return fig

# Create the chart
fig = create_static_position_changes_chart()

# Static configuration to remove interactivity and make it behave like an embedded image
static_config = {
    'displayModeBar': False,     # Remove toolbar completely
    'staticPlot': True,          # Make completely static - no zoom, pan, hover
    'responsive': False,         # Fixed size, not responsive
    'scrollZoom': False,         # Disable scroll zoom
    'doubleClick': False,        # Disable double-click actions
    'showTips': False,           # Disable tips
    'displaylogo': False,        # Remove Plotly logo
    'modeBarButtonsToRemove': ['zoom', 'pan', 'select', 'lasso', 'zoomIn', 'zoomOut', 'autoScale', 'resetScale']
}

# Display the static plot (will appear like an embedded image in HTML export)
fig.show(config=static_config)